<div align = "center">

# **Tarea 2: Conteo Aproximado**

</div>

## Librerias

In [120]:
import numpy as np
import numba
from numba import jit, prange
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import logsumexp
import pandas as pd
from tqdm import tqdm
import warnings
import os
import time
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configuración de semilla para reproducibilidad
np.random.seed(42)

# Crear directorio de resultados si no existe
os.makedirs('../results', exist_ok=True)
print("Directorio de resultados creado/verificado: ../results")

Directorio de resultados creado/verificado: ../results


## Funciones

### Parámetros según Teorema 9.1

In [121]:
# ============================================================================
# CÁLCULO DE PARÁMETROS
# ============================================================================

def calc_n_samples(K, q, epsilon):
    """
    Número mínimo de muestras por ratio.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    epsilon : float
        Precisión deseada
    
    Retorna:
    --------
    n_samples : int
    """
    # Fórmula simple basada en precisión
    return int(10_000 / epsilon)


def calc_n_steps_per_sample(K, q, epsilon):
    """
    Número mínimo de pasos del Gibbs sampler por muestra.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    epsilon : float
        Precisión deseada
    
    Retorna:
    --------
    n_steps : int
    """
    # Tiempo de mezcla: τ ≈ K² log(K)
    N = K * K
    tau_mix = int(N * np.log(N + 1))
    
    # Ajustar por q (más colores = más rápido)
    n_steps = int(tau_mix / np.log(q + 1))
    
    return max(100, n_steps)

In [122]:
# ============================================================================
# CONTEO DE q-COLORACIONES - VERSIÓN LIMPIA
# ============================================================================

def count_colorings(K, q, epsilon, max_steps_per_ratio):
    """
    Cuenta q-coloraciones en lattice K×K usando método telescópico.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    epsilon : float
        Precisión deseada
    max_steps_per_ratio : int
        Máximo de pasos totales del Gibbs sampler por ratio
    
    Retorna:
    --------
    result : dict
        - 'K': tamaño lattice
        - 'q': número de colores
        - 'epsilon': precisión usada
        - 'log_count': log(Z_{G,q})
        - 'count': Z_{G,q}
        - 'n_samples': muestras objetivo
        - 'n_steps': pasos por muestra
        - 'avg_ratio': ratio promedio de los k ratios
        - 'total_time': tiempo en segundos
    """
    # Calcular parámetros
    n_samples = calc_n_samples(K, q, epsilon)
    n_steps = calc_n_steps_per_sample(K, q, epsilon)
    
    # Crear aristas
    N = K * K
    all_edges = create_lattice_edges(K)
    k = len(all_edges)  # k = 2K(K-1)
    
    # Z_0 = q^(K²)
    log_Z_0 = N * np.log(q)
    
    # Estimar ratios
    log_product = 0.0
    ratios = []
    
    start_time = time.time()
    
    for i in range(1, k + 1):
        edges_i_minus_1 = all_edges[:i-1] if i > 1 else np.array([], dtype=np.int64).reshape(0, 4)
        edges_i = all_edges[:i]
        
        ratio, _, _ = estimate_ratio_telescopic_with_limit(
            K, edges_i_minus_1, edges_i, q,
            n_samples, n_steps,
            max_steps_total=max_steps_per_ratio,
            verbose=False
        )
        
        ratio_safe = max(ratio, 1e-300)
        log_product += np.log(ratio_safe)
        ratios.append(ratio)
    
    total_time = time.time() - start_time
    
    # Resultado final
    log_count = log_Z_0 + log_product
    count = np.exp(log_count) if log_count < 700 else np.inf
    
    return {
        'K': K,
        'q': q,
        'epsilon': epsilon,
        'log_count': log_count,
        'count': count,
        'n_samples': n_samples,
        'n_steps': n_steps,
        'avg_ratio': np.mean(ratios),
        'total_time': total_time
    }

In [123]:
# ============================================================================
# SCRIPT PARA EJECUTAR TODOS LOS EXPERIMENTOS
# ============================================================================

def run_all_experiments(K_min=3, K_max=20, q_min=2, q_max=15, 
                       epsilon=1.0,
                       max_steps_func=None,
                       results_dir='../results', 
                       checkpoint_file='checkpoint.csv'):
    """
    Ejecuta todos los experimentos para K ∈ [K_min, K_max] y q ∈ [q_min, q_max].
    
    Parámetros:
    -----------
    K_min, K_max : int
        Rango de tamaños de lattice
    q_min, q_max : int
        Rango de número de colores
    epsilon : float
        Precisión para todos los experimentos
    max_steps_func : callable, optional
        Función que recibe (K, q) y retorna max_steps_per_ratio.
        Si None, usa valores por defecto.
    results_dir : str
        Directorio para guardar resultados
    checkpoint_file : str
        Archivo CSV para checkpoints
    """
    # Función por defecto para max_steps
    if max_steps_func is None:
        def max_steps_func(K, q):
            if K <= 5:
                base = 20_000_000
            elif K <= 10:
                base = 10_000_000
            elif K <= 15:
                base = 5_000_000
            else:
                base = 2_000_000
            if q <= 3:
                return int(base * 1.5)
            elif q <= 6:
                return base
            else:
                return int(base * 0.7)
    
    os.makedirs(results_dir, exist_ok=True)
    checkpoint_path = os.path.join(results_dir, checkpoint_file)
    
    # Cargar previos
    completed = set()
    results = []
    if os.path.exists(checkpoint_path):
        df_prev = pd.read_csv(checkpoint_path)
        completed = set(zip(df_prev['K'], df_prev['q']))
        results = df_prev.to_dict('records')
        print(f"📂 {len(completed)} experimentos ya completados\n")
    
    # Calcular total
    K_values = list(range(K_min, K_max + 1))
    q_values = list(range(q_min, q_max + 1))
    total = len(K_values) * len(q_values)
    remaining = total - len(completed)
    
    print(f"{'='*70}")
    print(f"K ∈ [{K_min},{K_max}], q ∈ [{q_min},{q_max}], ε={epsilon}")
    print(f"Total: {total}, Completados: {len(completed)}, Pendientes: {remaining}")
    print(f"{'='*70}\n")
    
    if remaining == 0:
        print("✅ Completado")
        return pd.read_csv(checkpoint_path)
    
    count = len(completed)
    start_total = time.time()
    
    # Ejecutar experimentos
    for K in K_values:
        for q in q_values:
            if (K, q) in completed:
                continue
            
            count += 1
            max_steps = max_steps_func(K, q)
            
            print(f"[{count}/{total}] K={K:2d} q={q:2d} max_steps={max_steps:,} ... ", 
                  end='', flush=True)
            
            try:
                result = count_colorings(K, q, epsilon, max_steps)
                
                print(f"✓ log(Z)={result['log_count']:7.2f} "
                      f"Z≈{result['count']:.2e} "
                      f"t={result['total_time']:5.1f}s "
                      f"r̄={result['avg_ratio']:.3f}")
                
                results.append(result)
                pd.DataFrame(results).to_csv(checkpoint_path, index=False)
                
            except Exception as e:
                print(f"❌ {str(e)}")
    
    total_time = time.time() - start_total
    
    print(f"\n{'='*70}")
    print(f"Completado: {count}/{total}")
    print(f"Tiempo: {total_time:.1f}s ({total_time/60:.1f} min)")
    print(f"Guardado: {checkpoint_path}")
    print(f"{'='*70}\n")
    
    df_final = pd.DataFrame(results)
    final_path = os.path.join(results_dir, 'results_complete.csv')
    df_final.to_csv(final_path, index=False)
    
    return df_final

In [124]:
# ============================================================================
# ESTIMACIÓN DE RATIOS CON LÍMITE DE PASOS
# ============================================================================

def estimate_ratio_telescopic_with_limit(K, edges_i_minus_1, edges_i, q, 
                                         n_samples, n_steps_per_sample,
                                         max_steps_total=None,
                                         verbose=False):
    """
    Estima la razón r_i = Z_i / Z_{i-1} usando el método telescópico CON LÍMITE DE PASOS.
    
    Genera UNA muestra válida cada vez ejecutando n_steps_per_sample del Gibbs sampler.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    edges_i_minus_1 : ndarray de forma (i-1, 4)
        Aristas del grafo G_{i-1}
    edges_i : ndarray de forma (i, 4)
        Aristas del grafo G_i
    q : int
        Número de colores
    n_samples : int
        Número MÁXIMO de muestras para estimar el ratio
    n_steps_per_sample : int
        Pasos del Gibbs sampler para generar CADA muestra
    max_steps_total : int, optional
        Número MÁXIMO TOTAL de pasos del Gibbs sampler permitidos.
        Si None, se permite ejecutar todos los n_samples.
        Si se alcanza este límite, se usan las muestras colectadas hasta ese momento.
    verbose : bool
        Si True, imprime progreso cada cierto número de muestras
    
    Retorna:
    --------
    ratio : float
        Estimación del ratio r_i = Z_i / Z_{i-1}
    samples_collected : int
        Número real de muestras colectadas
    steps_executed : int
        Número total de pasos del Gibbs sampler ejecutados
    """
    # Inicializar coloración aleatoria de la lattice K×K
    coloring = np.random.randint(0, q, size=(K, K))
    
    # Colectar muestras y contar cuántas son válidas para G_i
    valid_count = 0
    samples_collected = 0
    steps_executed = 0
    
    print_interval = max(1, n_samples // 10)  # Imprimir cada 10% si verbose=True
    
    for sample_idx in range(n_samples):
        # Verificar si excedemos el límite de pasos
        if max_steps_total is not None:
            if steps_executed + n_steps_per_sample > max_steps_total:
                if verbose:
                    print(f"    ⚠️  Límite de pasos alcanzado: {steps_executed:,}/{max_steps_total:,} pasos. "
                          f"Muestras colectadas: {samples_collected:,}/{n_samples:,}")
                break
        
        # Generar UNA muestra ejecutando n_steps_per_sample del Gibbs sampler
        run_gibbs_sampler_partial(coloring, edges_i_minus_1, q, n_steps_per_sample)
        steps_executed += n_steps_per_sample
        
        # Verificar si la coloración también es válida para G_i
        if is_valid_coloring(coloring, edges_i):
            valid_count += 1
        
        samples_collected += 1
        
        # Imprimir progreso si verbose
        if verbose and (sample_idx + 1) % print_interval == 0:
            ratio_so_far = valid_count / samples_collected if samples_collected > 0 else 0
            print(f"    Progreso: {samples_collected:,}/{n_samples:,} muestras, "
                  f"ratio={ratio_so_far:.4f}, "
                  f"pasos={steps_executed:,}")
    
    # Estimar ratio
    if samples_collected == 0:
        ratio = 0.0
    else:
        ratio = valid_count / samples_collected
    
    return ratio, samples_collected, steps_executed

In [125]:
# ============================================================================
# REPRESENTACIÓN DE LA LATTICE K×K
# ============================================================================

def create_lattice_edges(K):
    """Crea aristas de lattice K×K con bordes libres."""
    edges = []
    # Aristas horizontales: (x,y) → (x+1,y)
    for y in range(K):
        for x in range(K - 1):
            edges.append((x, y, x+1, y))
    # Aristas verticales: (x,y) → (x,y+1)
    for y in range(K - 1):
        for x in range(K):
            edges.append((x, y, x, y+1))
    return np.array(edges, dtype=np.int64)


# ============================================================================
# GIBBS SAMPLER Y VALIDACIÓN
# ============================================================================

@jit(nopython=True)
def is_valid_coloring(coloring, edges):
    """Verifica si coloración es válida para grafo con aristas 'edges'."""
    for edge_idx in range(len(edges)):
        x1, y1, x2, y2 = edges[edge_idx]
        if coloring[x1, y1] == coloring[x2, y2]:
            return False
    return True


@jit(nopython=True)
def get_neighbor_colors(x, y, coloring, edges):
    """Obtiene colores de vecinos de (x,y) según aristas en 'edges'."""
    neighbor_colors = set()
    for edge_idx in range(len(edges)):
        x1, y1, x2, y2 = edges[edge_idx]
        if x1 == x and y1 == y:
            neighbor_colors.add(coloring[x2, y2])
        elif x2 == x and y2 == y:
            neighbor_colors.add(coloring[x1, y1])
    return neighbor_colors


@jit(nopython=True)
def gibbs_step_partial(coloring, edges, q):
    """
    Un paso del Gibbs sampler para grafo parcial.
    Muestrea sobre TODA la lattice K×K pero solo valida aristas en 'edges'.
    """
    K = coloring.shape[0]
    x = np.random.randint(0, K)
    y = np.random.randint(0, K)
    neighbor_colors = get_neighbor_colors(x, y, coloring, edges)
    valid_colors = []
    for c in range(q):
        if c not in neighbor_colors:
            valid_colors.append(c)
    if len(valid_colors) > 0:
        idx = np.random.randint(0, len(valid_colors))
        coloring[x, y] = valid_colors[idx]


@jit(nopython=True)
def run_gibbs_sampler_partial(coloring, edges, q, n_steps):
    """Ejecuta n_steps del Gibbs sampler."""
    for _ in range(n_steps):
        gibbs_step_partial(coloring, edges, q)

## Experimentos completos

In [126]:
# ============================================================================
# PRUEBA PEQUEÑA: K ∈ [3,5], q ∈ [2,5]
# ============================================================================

df_test = run_all_experiments(
    K_min=3, 
    K_max=3, 
    q_min=2, 
    q_max=2,
    epsilon=0.1,
    results_dir='../results',
    checkpoint_file='test.csv'
)

📂 12 experimentos ya completados

K ∈ [3,3], q ∈ [2,2], ε=0.1
Total: 1, Completados: 12, Pendientes: -11


Completado: 12/1
Tiempo: 0.0s (0.0 min)
Guardado: ../results/test.csv

